# Purpose

Demonstrate how to make SQL-based features testable and verifiable within Python client. See [client.py](client.py) for implementation.

# Sample data

The table looks like this:

| date | state | death | ... other columns |
| ---- | ----- | ----- | ----------------- |
| 2021-03-07 | DC | 1030 | ... |
| 2021-03-07 | NY | 39029 | ... |

To better understand the data, let's take a look at the sample data currently loaded in GBQ:

In [1]:
from google.cloud import bigquery

client = bigquery.Client()

QUERY = (
    """SELECT date, state, death, hospitalized 
         FROM `testable-features-poc.covid.us-states` 
        ORDER BY date DESC, state ASC
        LIMIT 5""")
query_job = client.query(QUERY)
rows = query_job.result()

for row in rows:
    print(row)

Row((datetime.date(2021, 3, 7), 'AK', 305, 1293), {'date': 0, 'state': 1, 'death': 2, 'hospitalized': 3})
Row((datetime.date(2021, 3, 7), 'AL', 10148, 45976), {'date': 0, 'state': 1, 'death': 2, 'hospitalized': 3})
Row((datetime.date(2021, 3, 7), 'AR', 5319, 14926), {'date': 0, 'state': 1, 'death': 2, 'hospitalized': 3})
Row((datetime.date(2021, 3, 7), 'AS', 0, None), {'date': 0, 'state': 1, 'death': 2, 'hospitalized': 3})
Row((datetime.date(2021, 3, 7), 'AZ', 16328, 57907), {'date': 0, 'state': 1, 'death': 2, 'hospitalized': 3})


# Define & validate feature
We'll define the feature in our notebook, and prior publishing it we'll validate it against production data.

Developers generally will not have direct access to production data on their laptops, so in practice, we'll probably need a service that proxies these requests between local environments and various prod and pre-prod environments.

In [2]:
from client import FeaturesClient, FeatureDefinition
import os

feature_def = FeatureDefinition("""
  sources:
    - name: source1
      environments:
        - name: prod
          value: testable-features-poc.covid.us-states
  query: |
    SELECT death
      FROM `{{ source1 }}`
     WHERE state = '{{ state }}'
     ORDER BY date DESC
     LIMIT 1
""")

c = FeaturesClient.load_feature(feature_def)

In [3]:
# connect to production environ to validate result
os.environ['ENV'] = 'prod'

# this should actually be a non-blocking LRO, like a future/promise...
c.inference(state='DC')    # expect 1030

1030

# Unit test

Now we'll write junit tests, enabling us to repeatedly validate our feature definition as well as test out edge cases.

In [4]:
from client import CSVSource

# Condition via CSV file.
#   Behind the scenes, this will load the CSV data into GBQ and create a reference to this table in feature definition

data = CSVSource("""
"date","state","death"
"2021-11-27","DC",123
"2021-11-27","VA",456
""")

c.condition_env('dev', 'source1', data)

In [5]:
# Switch to local development environment, which will use the data we conditioned
os.environ['ENV'] = 'dev'

# pytest goes here...
assert 123 == c.inference(state='DC')
assert None == c.inference(state='quebec')

print("tests passed.")

tests passed.


Note results are different for `dev` and `prod` environments. We can switch back and forth between these environments easily:

In [6]:
os.environ['ENV'] = 'prod'
assert 1030 == c.inference(state='DC')

os.environ['ENV'] = 'dev'
assert 123 == c.inference(state='DC')

print("tests passed.")

tests passed.


# Integration and E2E tests

This is equivalent to unit tests. You have three options for data sources:

1. If the environment's data is static and shared, you can define it in the feature definition, alongside the `prod` environment
2. You can considition it using CSV, as we did above with unit tests
3. You can specify a table source dynamically, which we'll do below

In [7]:
from client import TableSource

# temporary table for e2e test on ephemeral stack
data = TableSource("testable-features-poc.covid.us-states-0242ac130002")

c.condition_env('qa', 'source1', data)

In [8]:
os.environ['ENV'] = 'qa'

# ... tests run similar to above

# Publish feature

Now that we've validated and tested our feature definition, we can publish it -- either from our notebook, or better yet via CI/CD.

In [9]:
from client import FeatureRegistry

FeatureRegistry.publish(feature_def)

Once published, the feature definition is immutable. A client can fetch the definition via a handle (e.g., a UUID). Note this is equivalent to using the local definition, as this notebook did; this enables us to thoroughly test our feature prior to publishing, but providing portability.